In [41]:
import sys
import os
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter

In [38]:
path = "/Users/spencerbraun/Documents/Stanford/CS 230 - Deep Learning/Project/"
short_answer_file = path+ "asap-sas/train.tsv"

i=0

dataList = []
with open(short_answer_file) as f:
    for line in f:
        if line.startswith("Id"):
            continue
        splitLine = line.strip().split("\t")
        scores = map(int, splitLine[2:4])
        avgscore = sum(scores)/2
        essay = sent_tokenize(splitLine[4])
        for sent in essay:
            dataList.append([sent, avgscore])
    

In [39]:
highScores = []
lowScores = []
for data in dataList:
    if len(word_tokenize(data[0])) > 15:
        continue
    if data[1] >= 2:
        highScores.append([data[0]])
    if data[1] ==0:
        lowScores.append([data[0]])
    

In [40]:
import csv

with open('sas_high.csv', 'w', newline='\n') as f:
    writer = csv.writer(f)
    writer.writerows(highScores)
    

with open('sas_low.csv', 'w', newline='\n') as f:
    writer = csv.writer(f)
    writer.writerows(lowScores)

In [42]:
Counter([x[1] for x in dataList])

Counter({1.0: 11279,
         0.0: 11879,
         2.0: 11669,
         0.5: 2255,
         3.0: 2247,
         2.5: 510,
         1.5: 3216})

In [ ]:
def sentence_to_wordlist(raw_sentence):
    
    clean_sentence = re.sub("[^a-zA-Z0-9]"," ", raw_sentence)
    tokens = nltk.word_tokenize(clean_sentence)
    
    return tokens

In [ ]:

def tokenize(essay):
    stripped_essay = essay.strip()
    
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(stripped_essay)
    
    tokenized_sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            tokenized_sentences.append(sentence_to_wordlist(raw_sentence))
    
    return tokenized_sentences

In [47]:
clean_text(dataList[0][0])

'some additional information that we would need to replicate the experiment is how much vinegar should be placed in each identical container how or what tool to use to measure the mass of the four different samples and how much distilled water to use to rinse the four samples after taking them out of the vinegar'

In [ ]:
class Vocabulary(object):
    def __init__(self, vocab_file, emb_file='', dim_emb=0):
        with open(vocab_file, 'rb') as f:
            self.size, self.word2id, self.id2word = pickle.load(f)
        self.dim_emb = dim_emb
        self.embedding = np.random.random_sample(
            (self.size, self.dim_emb)) - 0.5

        if emb_file:
            print( 'Loading word vectors from', emb_file)
            with open(emb_file) as f:
                for line in f:
                    parts = line.split()
                    word = parts[0]
                    vec = np.array([float(x) for x in parts[1:]])
                    if word in self.word2id:
                        self.embedding[self.word2id[word]] = vec

        for i in range(self.size):
            self.embedding[i] /= LA.norm(self.embedding[i])

def build_vocab(data, path, min_occur=5):
    word2id = {'<pad>':0, '<go>':1, '<eos>':2, '<unk>':3}
    id2word = ['<pad>', '<go>', '<eos>', '<unk>']

    words = [word for sent in data for word in sent]
    cnt = Counter(words)
    for word in cnt:
        if cnt[word] >= min_occur:
            word2id[word] = len(word2id)
            id2word.append(word)
    vocab_size = len(word2id)
    with open(path, 'wb') as f:
        pickle.dump((vocab_size, word2id, id2word), f, pickle.HIGHEST_PROTOCOL)
